## 라이브러리 설치

In [250]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## 모델 호출

In [251]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("wumusill/final_project_kogpt2")
model = AutoModelForCausalLM.from_pretrained("wumusill/final_project_kogpt2")

## 삼행시 함수

In [252]:
import torch

# GPU 사용 여부
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

model.to(device)
model.eval()
print(device)

cuda:0


In [ ]:
def mind(input_letter):
    # 두음 법칙 사전
    dooeum = {"라":"나", "락":"낙", "란":"난", "랄":"날", "람":"남", "랍":"납", "랑":"낭", 
          "래":"내", "랭":"냉", "냑":"약", "략":"약", "냥":"양", "량":"양", "녀":"여", 
          "려":"여", "녁":"역", "력":"역", "년":"연", "련":"연", "녈":"열", "렬":"열", 
          "념":"염", "렴":"염", "렵":"엽", "녕":"영", "령":"영", "녜":"예", "례":"예", 
          "로":"노", "록":"녹", "론":"논", "롱":"농", "뢰":"뇌", "뇨":"요", "료":"요", 
          "룡":"용", "루":"누", "뉴":"유", "류":"유", "뉵":"육", "륙":"육", "륜":"윤", 
          "률":"율", "륭":"융", "륵":"늑", "름":"늠", "릉":"능", "니":"이", "리":"이", 
          "린":'인', '림':'임', '립':'입'}
    # 결과물을 담을 list
    res_l = []

    # 한 글자씩 인덱스와 함께 가져옴
    for idx, val in enumerate(input_letter):
        # 두음 법칙 적용
        if val in dooeum.keys():
            val = dooeum[val]

        times = 0
        while times < 3:
            # 만약 idx 가 0 이라면 == 첫 글자
            if idx == 0:
                # 첫 글자 인코딩
                input_ids = tokenizer.encode(
                val, add_special_tokens=False, return_tensors="pt")
                
                # 첫 글자 인코딩 값으로 문장 생성
                output_sequence = model.generate(
                    input_ids=input_ids.to(device), 
                    do_sample=True, max_length=42,
                    min_length=5, temperature=0.9, repetition_penalty=1.5,
                    no_repeat_ngram_size=2)
            
            # 첫 글자가 아니라면
            else:
                # 좀더 매끄러운 삼행시를 위해 이전 문장이랑 현재 음절 연결
                # 이후 generate 된 문장에서 이전 문장에 대한 데이터 제거
                link_with_pre_sentence = " ".join(res_l) + " " + val  
                # print(link_with_pre_sentence)

                # 연결된 문장을 인코딩
                input_ids = tokenizer.encode(
                link_with_pre_sentence, add_special_tokens=False, return_tensors="pt")
                print(input_ids)
                # 인코딩 값으로 문장 생성
                output_sequence = model.generate(
                    input_ids=input_ids.to(device), 
                    do_sample=True, max_length=42,
                    min_length=len_sequence + 5, temperature=0.9, repetition_penalty=1.5,
                    no_repeat_ngram_size=2)

            # 생성된 문장 리스트로 변환 (인코딩 되어있고, 생성된 문장 뒤로 padding 이 있는 상태)
            generated_sequence = output_sequence.tolist()[0]

            # padding index 앞까지 slicing 함으로써 padding 제거, padding이 없을 수도 있기 때문에 조건문 확인 후 제거
            if tokenizer.pad_token_id in generated_sequence:
                generated_sequence = generated_sequence[:generated_sequence.index(tokenizer.pad_token_id)]
            
            # 첫 글자가 아니라면, generate 된 음절만 결과물 list에 들어갈 수 있게 앞 문장에 대한 인코딩 값 제거
            # print(generated_sequence)
            if idx != 0:
                # 이전 문장의 길이 이후로 슬라이싱해서 앞 문장 제거
                generated_sequence = generated_sequence[len_sequence:]

                # 다음 음절을 위해 길이 갱신
                len_sequence += len(generated_sequence)        
            
            # 첫 글자라면
            else:
                # 시퀀스 길이 저장
                len_sequence = len(generated_sequence)

            # 결과물 디코딩
            decoded_sequence = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)

            if len(decoded_sequence) > 1:
                break
            else:
                times += 1
                continue

        # 결과물 리스트에 담기
        res_l.append(decoded_sequence)

        # print(res_l)

    # 결과물 list에서 한 줄씩 출력
    for letter, res in zip(input_letter, res_l):
        print(f"{letter} :", res)

In [319]:
def mind2(input_letter):
    # 두음 법칙 사전
    dooeum = {"라":"나", "락":"낙", "란":"난", "랄":"날", "람":"남", "랍":"납", "랑":"낭", 
          "래":"내", "랭":"냉", "냑":"약", "략":"약", "냥":"양", "량":"양", "녀":"여", 
          "려":"여", "녁":"역", "력":"역", "년":"연", "련":"연", "녈":"열", "렬":"열", 
          "념":"염", "렴":"염", "렵":"엽", "녕":"영", "령":"영", "녜":"예", "례":"예", 
          "로":"노", "록":"녹", "론":"논", "롱":"농", "뢰":"뇌", "뇨":"요", "료":"요", 
          "룡":"용", "루":"누", "뉴":"유", "류":"유", "뉵":"육", "륙":"육", "륜":"윤", 
          "률":"율", "륭":"융", "륵":"늑", "름":"늠", "릉":"능", "니":"이", "리":"이", 
          "린":'인', '림':'임', '립':'입'}
    # 결과물을 담을 list
    res_l = []

    # 한 글자씩 인덱스와 함께 가져옴
    for idx, val in enumerate(input_letter):
        # 두음 법칙 적용
        if val in dooeum.keys():
            val = dooeum[val]


        while True:
            # 만약 idx 가 0 이라면 == 첫 글자
            if idx == 0:
                # 첫 글자 인코딩
                input_ids = tokenizer.encode(
                val, add_special_tokens=False, return_tensors="pt")
                # print(f"{idx}번 인코딩 : {input_ids}\n") # 2차원 텐서

                # 첫 글자 인코딩 값으로 문장 생성
                output_sequence = model.generate(
                    input_ids=input_ids.to(device), 
                    do_sample=True, max_length=42,
                    min_length=5, temperature=0.9, repetition_penalty=1.5,
                    no_repeat_ngram_size=2)[0]
                # print("첫 글자 인코딩 후 generate 결과:", output_sequence, "\n") # tensor

            # 첫 글자가 아니라면
            else:
                # 한 음절
                input_ids = tokenizer.encode(
                val, add_special_tokens=False, return_tensors="pt")
                # print(f"{idx}번 째 글자 인코딩 : {input_ids} \n")

                # 좀더 매끄러운 삼행시를 위해 이전 인코딩과 지금 인코딩 연결
                link_with_pre_sentence = torch.cat((generated_sequence, input_ids[0]), 0)
                link_with_pre_sentence = torch.reshape(link_with_pre_sentence, (1, len(link_with_pre_sentence)))
                # print(f"이전 텐서와 연결된 텐서 {link_with_pre_sentence} \n")

                # 인코딩 값으로 문장 생성
                output_sequence = model.generate(
                    input_ids=link_with_pre_sentence.to(device), 
                    do_sample=True, max_length=42,
                    min_length=5, temperature=0.9, repetition_penalty=1.5,
                    no_repeat_ngram_size=2)[0]
                # print(f"{idx}번 인코딩 후 generate : {output_sequence}")
        
            # 생성된 문장 리스트로 변환 (인코딩 되어있고, 생성된 문장 뒤로 padding 이 있는 상태)
            generated_sequence = output_sequence.tolist()
            # print(f"{idx}번 인코딩 리스트 : {generated_sequence} \n")

            # padding index 앞까지 slicing 함으로써 padding 제거, padding이 없을 수도 있기 때문에 조건문 확인 후 제거
            if tokenizer.pad_token_id in generated_sequence:
                generated_sequence = generated_sequence[:generated_sequence.index(tokenizer.pad_token_id)]
            
            generated_sequence = torch.tensor(generated_sequence) 
            # print(f"{idx}번 인코딩 리스트 패딩 제거 후 다시 텐서 : {generated_sequence} \n")

            # 첫 글자가 아니라면, generate 된 음절만 결과물 list에 들어갈 수 있게 앞 문장에 대한 인코딩 값 제거
            # print(generated_sequence)
            if idx != 0:
                # 이전 문장의 길이 이후로 슬라이싱해서 앞 문장 제거
                generated_sequence = generated_sequence[len_sequence:]

            len_sequence = len(generated_sequence)
            # print("len_seq", len_sequence)

            # 음절 그대로 뱉으면 다시 해와, 아니면 while문 탈출
            if len_sequence > 1:
                break

        # 결과물 리스트에 담기
        res_l.append(generated_sequence)

        # print("res_l :", res_l)

    # 결과물 list에서 한 줄씩 출력
    for letter, res in zip(input_letter, res_l):
        decode_res = tokenizer.decode(res, clean_up_tokenization_spaces=True)
        print(f"{letter} :", decode_res)

In [320]:
mind2("어어어")

어 : 어서 와 내게 내 안에 숨겨둔 이게 사랑이라면
어 : 어떨까
어 : 어떡할까


In [321]:
mind2("고고고")

고 : 고맙다는 말도 하지 못하고
고 : 고마워서
고 : 고맙단 말도 못 했어


In [287]:
mind("해파리")

해 : 해줄게 없기에
파 : 파 무색해
리 : 이토록 예쁠까


In [288]:
mind("아이스크림")

아 : 아련히 들리네 눈 내리던 어느 날
이 : 이 거리에 낯선
스 : 스쳐 지나고
크 : 크면 잊혀질까요
림 : 임


In [284]:
mind("파이널")

파 : 파랑새 울어요
이 : 이 마음
널 : 널 향하는 내 맘 받아줘


In [274]:
mind("박경택")

박 : 박혀버린 깨진 꿈 조각
경 : 경적
택 : 택이


In [275]:
mind("문종현")

문 : 문득 생각이 난 거야
종 : 종잡치 못한
현 : 현길 홀로


In [276]:
mind("이정은")

이 : 이젠 내게서 떠나 가라고
정 : 정말로 괜찮나요
은 : 은마오


In [277]:
mind("권소희")

권 : 권태기란 말은 다
소 : 소용이 없어
희 : 희미한 시계소리만


In [278]:
mind("박건영")

박 : 박수를 치자
건 : 건네주겠지
영 : 영영 떠나갈꺼야


In [264]:
mind("정재영")

정 : 정말로 괜찮다니
재 : 재미가 없어
영 : 영영


In [265]:
mind("구자현")

구 : 구겨진 내 마음 알 때도
자 : 자꾸만
현 : 현상에 겨워


In [266]:
mind("김의준")

김 : 김건모 노래
의 : 의이지만
준 : 준 것 없는


In [267]:
mind("최지영")

최 : 최선이라고 생각했어
지 : 지치고 힘이 들 때
영 : 영원을 바래


In [268]:
mind("이지혜")

이 : 이젠 그만 너를 보내야 해
지 : 지겹게도 놓여진
혜 : 혜화원 아저씨


# 혹시 몰라 저장
* 와일문 추가 전 스트림릿

In [ ]:
import requests
import streamlit as st
from streamlit_lottie import st_lottie
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Page Config
st.set_page_config(
    page_title="노래 가사 n행시",
    page_icon="💌",
    layout="wide"
)

### Model
tokenizer = AutoTokenizer.from_pretrained("wumusill/final_20man")

@st.cache(show_spinner=False)
def load_model():
    model = AutoModelForCausalLM.from_pretrained("wumusill/final_20man")
    return model

model = load_model()

# Class : Dict 중복 키 출력
class poem(object):
    def __init__(self,letter):
        self.letter = letter

    def __str__(self):
        return self.letter

    def __repr__(self):
        return "'"+self.letter+"'"


def n_line_poem(input_letter):

    # 두음 법칙 사전
    dooeum = {"라":"나", "락":"낙", "란":"난", "랄":"날", "람":"남", "랍":"납", "랑":"낭", 
          "래":"내", "랭":"냉", "냑":"약", "략":"약", "냥":"양", "량":"양", "녀":"여", 
          "려":"여", "녁":"역", "력":"역", "년":"연", "련":"연", "녈":"열", "렬":"열", 
          "념":"염", "렴":"염", "렵":"엽", "녕":"영", "령":"영", "녜":"예", "례":"예", 
          "로":"노", "록":"녹", "론":"논", "롱":"농", "뢰":"뇌", "뇨":"요", "료":"요", 
          "룡":"용", "루":"누", "뉴":"유", "류":"유", "뉵":"육", "륙":"육", "륜":"윤", 
          "률":"율", "륭":"융", "륵":"늑", "름":"늠", "릉":"능", "니":"이", "리":"이", 
          "린":'인', '림':'임', '립':'입'}
    # 결과물을 담을 list
    res_l = []

    # 한 글자씩 인덱스와 함께 가져옴
    for idx, val in enumerate(input_letter):
        # 두음 법칙 적용
        if val in dooeum.keys():
            val = dooeum[val]

        # 만약 idx 가 0 이라면 == 첫 글자
        if idx == 0:
            # 첫 글자 인코딩
            input_ids = tokenizer.encode(
            val, add_special_tokens=False, return_tensors="pt")
            
            # 첫 글자 인코딩 값으로 문장 생성
            output_sequence = model.generate(
                input_ids=input_ids, 
                do_sample=True, max_length=42, no_repeat_ngram_size=2,
                min_length=5, temperature=0.9, repetition_penalty=1.5)
        
        # 첫 글자가 아니라면
        else:
            # 좀더 매끄러운 삼행시를 위해 이전 문장이랑 현재 음절 연결
            # 이후 generate 된 문장에서 이전 문장에 대한 데이터 제거
            link_with_pre_sentence = " ".join(res_l) + " " + val  
            # print(link_with_pre_sentence)

            # 연결된 문장을 인코딩
            input_ids = tokenizer.encode(
            link_with_pre_sentence, add_special_tokens=False, return_tensors="pt")

            # 인코딩 값으로 문장 생성
            output_sequence = model.generate(
                input_ids=input_ids, 
                do_sample=True, max_length=42, no_repeat_ngram_size=2,
                min_length=len_sequence, temperature=0.9, repetition_penalty=1.5)

        # 생성된 문장 리스트로 변환 (인코딩 되어있고, 생성된 문장 뒤로 padding 이 있는 상태)
        generated_sequence = output_sequence.tolist()[0]

        # padding index 앞까지 slicing 함으로써 padding 제거, padding이 없을 수도 있기 때문에 조건문 확인 후 제거
        if tokenizer.pad_token_id in generated_sequence:
            generated_sequence = generated_sequence[:generated_sequence.index(tokenizer.pad_token_id)]
        
        # 첫 글자가 아니라면, generate 된 음절만 결과물 list에 들어갈 수 있게 앞 문장에 대한 인코딩 값 제거
        # print(generated_sequence)
        if idx != 0:
            # 이전 문장의 길이 이후로 슬라이싱해서 앞 문장 제거
            generated_sequence = generated_sequence[len_sequence:]

            # 다음 음절을 위해 길이 갱신
            len_sequence += len(generated_sequence)        
        
        # 첫 글자라면
        else:
            # 시퀀스 길이 저장
            len_sequence = len(generated_sequence)
        
        # print(last_sequence)

        # 결과물 디코딩
        decoded_sequence = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)

        # 결과물 리스트에 담기
        res_l.append(decoded_sequence)

    poem_dict = {}

    for letter, res in zip(input_letter, res_l):
        poem_dict[poem(letter)] = res

    return poem_dict

###

# Image(.gif)
@st.cache(show_spinner=False)
def load_lottieurl(url: str):
    r = requests.get(url)
    if r.status_code != 200:
        return None
    return r.json()

lottie_url = "https://assets7.lottiefiles.com/private_files/lf30_fjln45y5.json"

lottie_json = load_lottieurl(lottie_url)
st_lottie(lottie_json, speed=1, height=200, key="initial")


# Title
row0_spacer1, row0_1, row0_spacer2, row0_2, row0_spacer3 = st.columns(
    (0.01, 2, 0.05, 1, 0.01)
)

with row0_1:
    st.title("한글 노래 가사 n행시")
    st.subheader("멋쟁이사자처럼 AIS7 파이널 프로젝트")

with row0_2:
    st.write("")
    st.subheader(
        "해파리팀"
    )
    st.write("이지혜, 최지영, 권소희")
    st.write("문종현, 구자현, 김의준")

st.write('---')

# Explanation
row1_spacer1, row1_1, row1_spacer2 = st.columns((0.01, 3, 0.01))

with row1_1:
    st.markdown(
        "**'MZ세대'에게**"
    )
    st.markdown(
        "음악은 세대를 드러내는 지표이자 자신의 감정 및 공동체를 드러내는 수단이다."
    )

st.write('---')

# Model & Input
row2_spacer1, row2_1, row2_spacer2 = st.columns((0.01, 1.5, 0.05))

# Word Input
if "generate" not in st.session_state:
    st.session_state.generate = False

with row2_1:
    word_input = st.text_input(
            "n행시에 사용할 단어를 적고 Enter를 눌러주세요. 👇",
            placeholder='한글 단어',
            max_chars=10
    )
    
    if word_input:
        st.write("n행시 단어 :  ", word_input)

    if st.button('n행시 제작하기'):
        with st.spinner('잠시 기다려주세요...'):
            result = n_line_poem(word_input)
        st.success('완료됐습니다!')
        for r in result:
            st.write(f'{r} : {result[r]}')

In [ ]:
from torch import tensor, cat, reshape
t1 = tensor([[ 9208,  6866, 12422, 31656,  7285,  9065]])
t2 = tensor([[ 9208,  6866, 12422, 31656,  7285,  9111]])

t = cat((t1[0], t2[0]), 0)
print(t)
print(len(t))
reshape(t, (1, 12))

tensor([ 9208,  6866, 12422, 31656,  7285,  9065,  9208,  6866, 12422, 31656,
         7285,  9111])
12


tensor([[ 9208,  6866, 12422, 31656,  7285,  9065,  9208,  6866, 12422, 31656,
          7285,  9111]])